## Bitcoin Allocation Strategies
Comparing different allocation strategies historically

. Lump-Sum

. Dollar Cost Averaging (DCA)

. Combination (optimized)

In [485]:
# Import Libraries
import pandas as pd
import numpy as np
import urllib
import requests
import riskfolio as rp
from datetime import datetime, timedelta
from copy import deepcopy

# Chart libraries + settings
%matplotlib inline
import matplotlib.pyplot as plt
# plt.style.use('seaborn-whitegrid')
pd.options.mode.chained_assignment = None  # default='warn' - disable some pandas warnings


In [384]:
# Load Bitcoin Prices into a dataframe
# Ticker is customizable
ticker = "BTC"
# Cryptocompare URL and fiels
base_url = 'https://min-api.cryptocompare.com/data/histoday'
ticker_field = 'fsym'
field_dict = {'tsym': 'USD','allData': 'true'}
# Convert the field dict into a url encoded string
url_args = "&" + urllib.parse.urlencode(field_dict)
ticker = ticker.upper()
globalURL = (base_url + "?" + ticker_field + "=" + ticker + url_args)


In [385]:
# Request the data
resp = requests.get(url=globalURL)
data = resp.json()
data["Response"]

'Success'

In [386]:
# Parse the JSON into a Pandas DataFrame
try:
    df = pd.DataFrame.from_dict(data['Data'])
    df = df.rename(columns={'time': 'date'})
    df['date'] = pd.to_datetime(df['date'], unit='s')
    df.set_index('date', inplace=True)
    df_save = df[['close', 'open', 'high', 'low']]
except Exception as e:
    print(e)
    df_save = None

In [387]:
# Include percentage change 
df = df_save
df['change'] = df['close'].pct_change()

### Support Utilities to be used later

In [388]:
# Increment n number of months of certain date
def monthdelta(date, delta):
    m, y = (date.month+delta) % 12, date.year + ((date.month)+delta-1) // 12
    if not m: m = 12
    d = min(date.day, [31,
        29 if y%4==0 and not y%400==0 else 28,31,30,31,30,31,31,30,31,30,31][m-1])
    new_date = (date.replace(day=d,month=m, year=y))
    return new_date

def add_periods(date, periods, frequency):
    if frequency.upper() == 'D' or 'DAY' in frequency.upper():
        return (date + timedelta(days=periods))
    if frequency.upper() == 'W' or 'WEEK' in frequency.upper():
        return (date + timedelta(days=periods * 7))
    if frequency.upper() == 'M' or 'MONTH' in frequency.upper():
        return(monthdelta(date, periods))
    if frequency.upper() == 'Y' or 'YEAR' in frequency.upper():
        return(monthdelta(date, periods * 12))
    
def annualization_factor(df):
    """
    Receives a df and returns the number of periods to apply
    to annualize the returns. For BTC this should be close to
    365 as it trades daily. For stocks should be close to 252.
    Args:
        df (_type_): _description_
    """
    start_date = df.index[0]
    end_date = df.index[-1]
    number_of_days = (end_date - start_date).days
    fraction_of_year = number_of_days / 365
    data_points = len(df)
    annualization_factor = data_points / fraction_of_year
    return int(round(annualization_factor, 0))


### Main Allocation Class
See example on creating an allocation instance at the cell following the class definition

In [448]:
class AllocationManager:
    def __init__(self):
        self.frequency = 'D'  # 'D', 'W', 'M', 'Y'
        self.allocation_periods = 30  # Assume allocation happens during 30 periods
        self.upfront_percent = 0  # [0 - 1]: amount to be allocated upfront
        self.capital = 100000  # 10,000 dollars to allocate
        self.df = df  # Bitcoin Prices Dataframe
        self.start_date = self.df.index.min()  # Date where allocation starts (default = first date)
                                               # ex: pd.to_datetime('2023-01-01', format='%Y-%m-%d')
        self.end_date = self.df.index.max()  # End date for analysis (default = today), but this can be used to test specific timeframes (ex: ending last year)
                                             # ex: pd.to_datetime('2023-01-01', format='%Y-%m-%d')
        # Create empty allocation & position columns
        self.df['allocation'] = 0
        self.df['BTC_tx'] = 0
        self.risk_free_rate = 0.05
         
    def allocate_capital(self):
        # TRIM THE DF between start and end dates
        # Filter the dataframe to only include selected dates
        self.df = self.df[(self.df.index >= (self.start_date)) & (self.df.index <= self.end_date)]
        
        # Updates the dataframe to allocate the capital
        available_capital = self.capital 
        current_date = self.start_date
        periods_left = self.allocation_periods
        
        # Set upfront amount if any & per period amounts
        if self.upfront_percent > 0:
            upfront = self.upfront_percent * self.capital  # how much upfront in $
            per_period = (self.capital - upfront) / (self.allocation_periods - 1)
        else:
            per_period = self.capital / self.allocation_periods 
            upfront = per_period
        
        # Start looping until allocation is complete
        while periods_left > 0:
            # Allocate Capital
            if current_date == self.start_date:
                self.df.at[current_date, 'allocation'] = upfront
            else:
                self.df.at[current_date, 'allocation'] = per_period
            
            # Allocate BTC
            self.df.at[current_date, 'BTC_tx'] = (
                self.df.at[current_date, 'allocation'] / 
                self.df.at[current_date, 'close'] 
                )
            current_date = add_periods(current_date, 1, self.frequency)
            if current_date > self.end_date:
                raise Exception("Allocation dates overflow end date. Either shorten the allocation period or increase the data range.")
            periods_left -= 1

        # Sum all BTC Txs and calculate portfolio values
        self.df['BTC_position'] = self.df['BTC_tx'].cumsum()
        self.df['portfolio_position'] = (self.df['BTC_position'] * self.df['close'])
        self.df = self.df.fillna(0)
        # TO DO ---------------------
        self.df['cum_capital'] = self.df['allocation'].cumsum()
        self.df['normalized_port_position'] = (self.df['portfolio_position'] / 
                                               self.df['cum_capital']) * 100
        self.df['cum_return'] = self.df['normalized_port_position'] / 100

    def show_allocations(self):
        al_df = self.df.where(self.df.allocation > 0).dropna()
        return (al_df)
        
    def stats(self):
        df = self.df
        stats = {}
        stats = stats | deepcopy(self.__dict__)
        del stats['df'] 
        stats['capital allocated'] = df.allocation.sum()
        stats['BTC allocated'] = df.BTC_tx.sum()
        stats['max portfolio value'] = df.portfolio_position.max()
        stats['final portfolio value'] = df.portfolio_position[-1]
        # Calculate Return on Invested Capital
        stats['ROIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated']) - 1
            )
        # Calculate Multiple of Invested Capital
        stats['MOIC'] = (
            (stats['final portfolio value'] / 
             stats['capital allocated'])
            )
        
        stats['number_points'] = df['change'].count()
        stats['volatility_daily'] = df['change'].std()
        stats['annualization_factor'] = annualization_factor(df)
        stats['volatility_annual'] = df['change'].std() * annualization_factor(df)**.5
        stats['return_annual'] = ((df['cum_return'][-1]) ** (annualization_factor(df) / df['change'].count())) -1
    
        stats['sharpe_ratio'] = (stats['return_annual'] -
                             self.risk_free_rate) / stats['volatility_annual']
        Y = df['normalized_port_position'].pct_change().dropna()
        stats['hist_VaR_95'] = rp.RiskFunctions.VaR_Hist(Y, alpha=0.05)
        stats['max_DD_abs'] = rp.RiskFunctions.MDD_Rel(Y)
        stats['avg_DD_abs'] = rp.RiskFunctions.ADD_Rel(Y)
        stats['DaR_DD_abs_95'] = rp.RiskFunctions.DaR_Rel(Y, alpha=0.05)
        # Bitcoin stats
        stats['BTC_initial'] = df['close'][0]
        stats['BTC_final'] = df['close'][-1]
        stats['BTC_MOIC'] = df['close'][-1] / df['close'][0]
        stats['BTC_period'] = (stats['BTC_final'] / stats['BTC_initial']) - 1
        
        # Compared to BTC - how much more or less $100 invested in the 
        # allocation strategy resulted compared to $100 in BTC
        stats['outperformance_DV100'] = (stats['MOIC'] * 100) - (stats['BTC_MOIC'] * 100)
        
        #  Cost Basis Statistics
        stats['weighted_avg_cost'] =  (stats['capital allocated'] / 
                                       stats['BTC allocated'])
        txs = self.show_allocations()
        stats['avg_buy_price'] = txs['close'].mean()
        stats['max_buy_price'] = txs['close'].max()
        stats['min_buy_price'] = txs['close'].min()
        stats['first_buy_price'] = txs['close'][0]
        stats['lump_sum_beats_avg'] = (stats['first_buy_price'] < stats['avg_buy_price'])
        return (stats)
    
def run_through_time(alloc):
    # Do the same allocation for every start date 
    # in the dataframe. This basically shortens the dataframe by 1 day
    # on every loop and stores the results.
    # And stores the results in a new df with stats.
    
    # Stores initial values
    stats = {}
    stats['inputs'] = stats | deepcopy(alloc.__dict__)
    
    # Store allocations outputs
    stats['outputs'] = {}
    # Remove the df to save memory
    del stats['inputs']['df'] 
    tmp_alloc = deepcopy(alloc)  # Creates a copy of Alloc Instance

    results = []    
    # Will loop from start date until there aren't enough periods to allocate 
    while True:
        try:
            tmp_alloc.allocate_capital()
            results.append(tmp_alloc.stats())
            tmp_alloc.start_date = add_periods(tmp_alloc.start_date, 1, 'D')
        except Exception as e:
            break
    df = pd.DataFrame(results)
    
    # Create Outputs - sums, averages, depending on column
    stats['outputs']['number_of_allocations'] = df['frequency'].count()
    stats['outputs']['avg_BTC_allocated'] = df['BTC allocated'].mean()
    stats['outputs']['avg_lump_sum_beats_avg'] = df['lump_sum_beats_avg'].mean()
    stats['outputs']['avg_ROIC'] = df['ROIC'].mean()
    stats['outputs']['avg_sharpe'] = df['sharpe_ratio'].mean()
    stats['outputs']['avg_DD'] = df['avg_DD_abs'].mean()
    stats['outputs']['avg_DaR_DD_abs_95'] = df['DaR_DD_abs_95'].mean()
    stats['outputs']['avg_outperformance_DV100'] = df['outperformance_DV100'].mean()
    
    return (df, stats)
        
        

In [456]:
# Create a sample Instance of the AllocationManager and test results
btc_alloc = AllocationManager()
btc_alloc.capital = 100
btc_alloc.allocation_periods = 8
btc_alloc.frequency = 'W'
btc_alloc.start_date = pd.to_datetime('2017-01-01', format='%Y-%m-%d')
btc_alloc.upfront_percent = 0.50
btc_alloc.allocate_capital()

In [457]:
# Show statistics
btc_alloc.stats()

{'frequency': 'W',
 'allocation_periods': 8,
 'upfront_percent': 0.5,
 'capital': 100,
 'start_date': Timestamp('2017-01-01 00:00:00'),
 'end_date': Timestamp('2023-01-19 00:00:00'),
 'risk_free_rate': 0.05,
 'capital allocated': 100.0,
 'BTC allocated': 0.10344771670767026,
 'max portfolio value': 6987.804298566757,
 'final portfolio value': 2142.998071864087,
 'ROIC': 20.429980718640866,
 'MOIC': 21.429980718640866,
 'number_points': 2210,
 'volatility_daily': 0.04113098975427643,
 'annualization_factor': 365,
 'volatility_annual': 0.7858064558978459,
 'return_annual': 0.6589350730308987,
 'sharpe_ratio': 0.7749173711421629,
 'hist_VaR_95': 0.06342822632250167,
 'max_DD_abs': 0.8329062742789147,
 'avg_DD_abs': 0.42765312269721695,
 'DaR_DD_abs_95': 0.7913493015684795,
 'BTC_initial': 995.44,
 'BTC_final': 20715.76,
 'BTC_MOIC': 20.810656594068952,
 'BTC_period': 19.810656594068952,
 'outperformance_DV100': 61.93241245719173,
 'weighted_avg_cost': 966.6718916821233,
 'avg_buy_price': 

In [458]:
btc_alloc.df

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2017-01-01,995.44,963.38,1001.61,956.10,0.033279,50.0,0.050229,0.050229,50.000000,50.0,100.000000,1.000000
2017-01-02,1017.05,995.44,1031.68,990.20,0.021709,0.0,0.000000,0.050229,51.085450,50.0,102.170899,1.021709
2017-01-03,1033.30,1017.05,1035.47,1006.53,0.015978,0.0,0.000000,0.050229,51.901672,50.0,103.803343,1.038033
2017-01-04,1135.41,1033.30,1148.54,1022.32,0.098819,0.0,0.000000,0.050229,57.030559,50.0,114.061119,1.140611
2017-01-05,989.35,1135.41,1150.63,874.53,-0.128641,0.0,0.000000,0.050229,49.694105,50.0,99.388210,0.993882
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-01-15,20878.94,20954.52,21047.91,20575.44,-0.003607,0.0,0.000000,0.103448,2159.878670,100.0,2159.878670,21.598787
2023-01-16,21188.92,20878.94,21438.83,20635.89,0.014847,0.0,0.000000,0.103448,2191.945394,100.0,2191.945394,21.919454
2023-01-17,21136.12,21188.92,21555.18,20866.41,-0.002492,0.0,0.000000,0.103448,2186.483354,100.0,2186.483354,21.864834


In [478]:
d, stats = run_through_time(btc_alloc)
stats

{'inputs': {'frequency': 'W',
  'allocation_periods': 8,
  'upfront_percent': 0.5,
  'capital': 100,
  'start_date': Timestamp('2017-01-01 00:00:00'),
  'end_date': Timestamp('2023-01-19 00:00:00'),
  'risk_free_rate': 0.05},
 'outputs': {'number_of_allocations': 2154,
  'avg_BTC_allocated': 0.06033140059006965,
  'avg_lump_sum_beats_avg': 0.5552460538532962,
  'avg_ROIC': 2.1506110349520573,
  'avg_sharpe': 0.14720803012901137,
  'avg_DD': 0.358007624673099,
  'avg_DaR_DD_abs_95': 0.6908701620820283,
  'avg_outperformance_DV100': -21.66043860560593}}

In [447]:
d

Index(['frequency', 'allocation_periods', 'upfront_percent', 'capital',
       'start_date', 'end_date', 'risk_free_rate', 'capital allocated',
       'BTC allocated', 'max portfolio value', 'final portfolio value', 'ROIC',
       'MOIC', 'number_points', 'volatility_daily', 'annualization_factor',
       'volatility_annual', 'return_annual', 'sharpe_ratio', 'hist_VaR_95',
       'max_DD_abs', 'avg_DD_abs', 'DaR_DD_abs_95', 'BTC_initial', 'BTC_final',
       'BTC_MOIC', 'BTC_period', 'outperformance_DV100', 'weighted_avg_cost',
       'avg_buy_price', 'max_buy_price', 'min_buy_price', 'first_buy_price',
       'lump_sum_beats_avg'],
      dtype='object')

In [460]:
# Show only the allocation periods
al_df = btc_alloc.show_allocations()
display(al_df)
# Check it adds to allocation amount
print("Total allocation:")
print(round(al_df.allocation.sum(), 8))
if (round(al_df.allocation.sum(), 8) == round(btc_alloc.capital, 8)):
    print ("Checks [OK]")
else:
    print ("[ERROR] - something went wrong")

,close,open,high,low,change,allocation,BTC_tx,BTC_position,portfolio_position,cum_capital,normalized_port_position,cum_return
date,,,,,,,,,,,,
2017-01-01,995.44,963.38,1001.61,956.10,0.033279,50.000000,0.050229,0.050229,50.000000,50.000000,100.000000,1.000000
2017-01-08,900.86,888.87,936.13,875.85,0.013489,7.142857,0.007929,0.058158,52.392194,57.142857,91.686340,0.916863
2017-01-15,821.17,819.63,826.43,808.63,0.001879,7.142857,0.008698,0.066856,54.900442,64.285714,85.400688,0.854007
2017-01-22,918.84,919.84,938.38,888.62,-0.001087,7.142857,0.007774,0.074630,68.573161,71.428571,96.002425,0.960024
2017-01-29,914.55,918.51,922.40,912.68,-0.004311,7.142857,0.007810,0.082440,75.395854,78.571429,95.958360,0.959584
2017-02-05,1016.11,1031.85,1033.66,1004.05,-0.015254,7.142857,0.007030,0.089470,90.911357,85.714286,106.063250,1.060632
2017-02-12,996.52,1008.32,1007.82,992.45,-0.011703,7.142857,0.007168,0.096638,96.301497,92.857143,103.709304,1.037093
2017-02-19,1048.89,1052.28,1056.48,1037.67,-0.003222,7.142857,0.006810,0.103448,108.505276,100.000000,108.505276,1.085053


Total allocation:
100.0
Checks [OK]


### Run Simulations
Create different allocation scenarios

In [498]:
# CONSTANTS
# Define some static variables that will remain constant

# --------------
# Start Date
# --------------
# Data from 2010 has less meaning than recent data. For this analysis we can use
# more recent data. Using only data since 2017.
sim_start_date = pd.to_datetime('2022-01-01', format='%Y-%m-%d')

# Assuming $100,000 to be allocated
sim_allocated_capital = 100000

# Risk Free Rate (used to calculate sharpe ratio)
sim_risk_free_rate = 0.05


In [499]:
# Variables
# These will be the ranges / variables to be simulated

sim_frequencies = ['D', 'W', 'M']
sim_allocation_periods = {
    'D': range(2, 7),
    'W': range(2, 8),
    'M': range(2, 6)
}
sim_upfront_percents = [0, 0.25, 0.5, 0.75, 1]

In [474]:
# Loop to gather results
print("Running scenarios...")
sim_results =  []
for freq in sim_frequencies:
    for all in sim_allocation_periods[freq]:
        for upfr in sim_upfront_percents:
            # Create allocation instance
            sim_alloc = AllocationManager()
            sim_alloc.capital = sim_allocated_capital
            sim_alloc.allocation_periods = all
            sim_alloc.frequency = freq
            sim_alloc.start_date = sim_start_date
            sim_alloc.upfront_percent = upfr
            sim_alloc.allocate_capital()
            print('Running scenario: ' + str(freq) + ' ' + str(upfr) + ' ' + str(all))
            # Run this allocation through time
            _, stats = run_through_time(sim_alloc)
            sim_results.append(stats['inputs'] | stats['outputs'])

sim_df = pd.DataFrame(sim_results)

Running scenarios...
Running scenario: D 0 2
Running scenario: D 0.25 2
Running scenario: D 0.5 2
Running scenario: D 0.75 2
Running scenario: D 1 2
Running scenario: D 0 3
Running scenario: D 0.25 3
Running scenario: D 0.5 3
Running scenario: D 0.75 3
Running scenario: D 1 3
Running scenario: D 0 4
Running scenario: D 0.25 4
Running scenario: D 0.5 4
Running scenario: D 0.75 4
Running scenario: D 1 4
Running scenario: W 0 2
Running scenario: W 0.25 2
Running scenario: W 0.5 2
Running scenario: W 0.75 2
Running scenario: W 1 2
Running scenario: W 0 3
Running scenario: W 0.25 3
Running scenario: W 0.5 3
Running scenario: W 0.75 3
Running scenario: W 1 3
Running scenario: W 0 4
Running scenario: W 0.25 4
Running scenario: W 0.5 4
Running scenario: W 0.75 4
Running scenario: W 1 4
Running scenario: W 0 5
Running scenario: W 0.25 5
Running scenario: W 0.5 5
Running scenario: W 0.75 5
Running scenario: W 1 5
Running scenario: W 0 6
Running scenario: W 0.25 6
Running scenario: W 0.5 6
Runnin

In [476]:
sim_results

[{'frequency': 'D',
  'allocation_periods': 2,
  'upfront_percent': 0,
  'capital': 100000,
  'start_date': Timestamp('2022-01-01 00:00:00'),
  'end_date': Timestamp('2023-01-19 00:00:00'),
  'risk_free_rate': 0.05,
  'number_of_allocations': 382,
  'avg_BTC_allocated': 4.089539569440217,
  'avg_lump_sum_beats_avg': 0.468586387434555,
  'avg_ROIC': -0.15282079768973136,
  'avg_sharpe': 21.410751254715212,
  'avg_DD': 0.2534410998017287,
  'avg_DaR_DD_abs_95': 0.37216648707948996,
  'avg_outperformance_DV100': 0.07432597587932874},
 {'frequency': 'D',
  'allocation_periods': 2,
  'upfront_percent': 0.25,
  'capital': 100000,
  'start_date': Timestamp('2022-01-01 00:00:00'),
  'end_date': Timestamp('2023-01-19 00:00:00'),
  'risk_free_rate': 0.05,
  'number_of_allocations': 382,
  'avg_BTC_allocated': 4.091333517040482,
  'avg_lump_sum_beats_avg': 0.468586387434555,
  'avg_ROIC': -0.1524491678103347,
  'avg_sharpe': 19.357971812901827,
  'avg_DD': 0.2529921467364771,
  'avg_DaR_DD_abs_95

In [477]:
sim_df

,frequency,allocation_periods,upfront_percent,capital,start_date,end_date,risk_free_rate,number_of_allocations,avg_BTC_allocated,avg_lump_sum_beats_avg,avg_ROIC,avg_sharpe,avg_DD,avg_DaR_DD_abs_95,avg_outperformance_DV100
0,D,2,0.00,100000,2022-01-01,2023-01-19,0.05,382,4.089540,0.468586,-0.152821,21.410751,0.253441,0.372166,7.432598e-02
1,D,2,0.25,100000,2022-01-01,2023-01-19,0.05,382,4.091334,0.468586,-0.152449,19.357972,0.252992,0.371877,1.114890e-01
2,D,2,0.50,100000,2022-01-01,2023-01-19,0.05,382,4.089540,0.468586,-0.152821,21.410751,0.253441,0.372166,7.432598e-02
3,D,2,0.75,100000,2022-01-01,2023-01-19,0.05,382,4.087746,0.468586,-0.153192,24.209843,0.254092,0.372680,3.716299e-02
4,D,2,1.00,100000,2022-01-01,2023-01-19,0.05,382,4.085952,0.000000,-0.153564,28.158693,0.254954,0.373382,-5.208167e-16
5,D,3,0.00,100000,2022-01-01,2023-01-19,0.05,381,4.091253,0.459318,-0.152466,16.750995,0.252823,0.372123,1.449007e-01
6,D,3,0.25,100000,2022-01-01,2023-01-19,0.05,381,4.092127,0.459318,-0.152285,16.064320,0.252550,0.371926,1.630133e-01
7,D,3,0.50,100000,2022-01-01,2023-01-19,0.05,381,4.089504,0.459318,-0.152828,18.431874,0.253407,0.372553,1.086755e-01
8,D,3,0.75,100000,2022-01-01,2023-01-19,0.05,381,4.086881,0.459318,-0.153372,22.108965,0.254429,0.373372,5.433777e-02
9,D,3,1.00,100000,2022-01-01,2023-01-19,0.05,381,4.084258,0.000000,-0.153915,28.241798,0.255569,0.374305,-5.594825e-16
